In [1]:
!pip install datasets
!pip install transformers
!pip install seqeval

In [2]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer

dataset = load_dataset("conll2003")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
metric = load_metric("seqeval")

Reusing dataset conll2003 (/root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if True else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [4]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)
tokenized_datasets

Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee/cache-bb8c4b46d64ce3c6.arrow


  0%|          | 0/4 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee/cache-b35a419ad8d936ad.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3251
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3454
    })
})

In [5]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

label_list = dataset["train"].features[f"ner_tags"].feature.names
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_list))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

In [7]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [8]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [6]:
args = TrainingArguments(
    "BERT-finetuned-NER",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [9]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [10]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: pos_tags, tokens, ner_tags, chunk_tags, id.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14042
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1317


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.114547,0.847884,0.863280,0.855513,0.968303
2,0.221600,0.110874,0.870798,0.885474,0.878075,0.971911
3,0.059600,0.112867,0.875029,0.889153,0.882034,0.972844


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: pos_tags, tokens, ner_tags, chunk_tags, id.
***** Running Evaluation *****
  Num examples = 3454
  Batch size = 32
Saving model checkpoint to BERT-finetuned-NER/checkpoint-500
Configuration saved in BERT-finetuned-NER/checkpoint-500/config.json
Model weights saved in BERT-finetuned-NER/checkpoint-500/pytorch_model.bin
tokenizer config file saved in BERT-finetuned-NER/checkpoint-500/tokenizer_config.json
Special tokens file saved in BERT-finetuned-NER/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: pos_tags, tokens, ner_tags, chunk_tags, id.
***** Running Evaluation *****
  Num examples = 3454
  Batch size = 32
Saving model checkpoint to BERT-finetuned-NER/checkpoint-1000
Configuration saved in BERT-f

TrainOutput(global_step=1317, training_loss=0.11686755059429797, metrics={'train_runtime': 459.6281, 'train_samples_per_second': 91.652, 'train_steps_per_second': 2.865, 'total_flos': 566351299470348.0, 'train_loss': 0.11686755059429797, 'epoch': 3.0})

# Evaluate

In [11]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: pos_tags, tokens, ner_tags, chunk_tags, id.
***** Running Prediction *****
  Num examples = 3251
  Batch size = 32


{'LOC': {'f1': 0.9492231906025009,
  'number': 2618,
  'precision': 0.9417293233082706,
  'recall': 0.9568372803666921},
 'MISC': {'f1': 0.7860855657736904,
  'number': 1231,
  'precision': 0.7740157480314961,
  'recall': 0.7985377741673436},
 'ORG': {'f1': 0.882943143812709,
  'number': 2056,
  'precision': 0.8676056338028169,
  'recall': 0.8988326848249028},
 'PER': {'f1': 0.9756338491932829,
  'number': 3034,
  'precision': 0.9746710526315789,
  'recall': 0.9765985497692815},
 'overall_accuracy': 0.9816194576389661,
 'overall_f1': 0.9201175231442984,
 'overall_precision': 0.911978021978022,
 'overall_recall': 0.9284036245665063}